# Week 9 - Macro Facility Basics (Part 7)

*  %INCLUDE (a global SAS statement, not a macro statement)
* Compiling and saving macros
* Accessing Macros

[Jupyter Python Notebook Keyboard Shortcuts and Text Snippets for Beginners](http://maxmelnick.com/2016/04/19/python-beginner-tips-and-tricks.html)

In the following code snippet, 

* %INCLUDE brings in SAS statements stored in external files (part1, part2, and part3)
* the SOURCE2 option display interts SAS statement in SAS log

In [4]:
*Ex17_percent_include.sas;
options nodate nonumber ;
filename jobs "c:\sascourse\Week9";
%include jobs(Part1)/source2;
%include jobs(Part2)/source2;
%include jobs(Part3)/source2;
title;


Obs,TEST1,TEST2,TEST3,TEST4,TEST5,ASSIGNMENT1,ASSIGNMENT2,MIDTERM,FINAL,id
1,75,42,44,63,60,62,60,50,43,GWa04e20
2,44,96,53,69,99,85,91,60,89,GW42a7fe
3,41,66,95,46,44,48,83,84,96,GWf5a0a0
4,85,71,96,68,96,66,100,78,58,GW1d3e0a
5,75,96,96,78,73,98,86,70,94,GW95cd76
6,49,84,87,98,61,88,95,95,98,GW479e08
7,85,43,83,90,88,45,45,44,88,GW57926a
8,73,46,58,59,98,74,80,82,98,GW4290e1
9,80,79,88,51,84,62,48,96,95,GW00c6fd
10,97,57,99,72,59,96,97,93,49,GW1db6d4


In [2]:
%showLog

In [15]:
*Ex18_Macro_Include.sas;
options symbolgen;
*'Macro_List_Files.txt' includes a macro program authored by SAS(R) Institute;
%include 'c:\SASCourse\Week9\Macro_List_Files.txt'; 
ods Excel file="C:\SASCourse\Week9\List_SAS_Files.xlsx" ;
%macro mexcel (start, stop); 
%DO num = &START %TO &STOP;
      ods excel options(sheet_name="Week&num");
        data _null_;                                                                                                                            
         file print;                                                                                                                 
         %drive(C:\SASCourse\Week&num,sas)                                                                                                                   
         run;  
%end; 
ods Excel close; 
%mend mexcel;
/* Create an Excel Workbook with 2 sheets, each listing  
   all SAS files from Week9 and Week10 folders, respectively*/
%mexcel (9,10)


### Saving Macros Using the Stored Compiled Macro Facility

* Macros can be compiled and then stored temporarily or permanently for later use.

* When you compile a macro, it is usually stored in the temporary WORK.SASMACR catalog.  

* You can also store a macro in compiled form in a SAS macro catalog also named SASMACR, but in a permanent library


In [13]:
*Ex19_Compiled_Macro.sas;
options nodate nonumber nosource;
ods html close;
*** This is a revised version of the macro obtained from 
    http://support.sas.com/kb/45/805.html
    [Sample 45805: List all files within a directory including 
     sub-directories]
    Revisions: Macro calls are wrapped in data steps; 

/* The following two lines and the options to the macro
   are added by Pradip Muhuri 
 */
Libname MyLib 'C:\SASCourse\Compiled_Macros';
options mstored sasmstore=MyLib;
%macro drive(dir,ext) /store source
        des='Listing SAS Files';  
  /* Defime macro variables */ 
  %local filrf rc did memcnt name i;                                                                                                    
                                                                                                                                        
  /* Assigns a fileref to the directory and opens the directory */                                                                      
  %let rc=%sysfunc(filename(filrf,&dir));                                                                                               
  %let did=%sysfunc(dopen(&filrf));                                                                                                     
                                                                                                                                        
  /* Make sure directory can be open */                                                                                                 
  %if &did eq 0 %then %do;                                                                                                              
   %put Directory &dir cannot be open or does not exist;                                                                                
   %return;                                                                                                                             
  %end;                                                                                                                                 
   /* Loops through entire directory */                                                                                                 
   %do i = 1 %to %sysfunc(dnum(&did));                                                                                                  
                                                                                                                                        
     /* Retrieve name of each file */                                                                                                   
     %let name=%qsysfunc(dread(&did,&i));                                                                                               
                                                                                                                                        
     /* Checks to see if the extension matches the parameter value */                                                                   
     /* If condition is true print the full name to the log        */                                                                   
      %if %qupcase(%qscan(&name,-1,.)) = %upcase(&ext) %then %do;                                                                       
        put "&dir\&name";                                                                                                               
      %end;                                                                                                                             
     /* If directory name call macro again */                                                                                           
      %else %if %qscan(&name,2,.) = %then %do;                                                                                          
        %drive(&dir\%unquote(&name),&ext)                                                                                               
      %end;                                                                                                          
   %end;                                                                                                                                
  /* Closes the directory and clear the fileref */                                                                                      
  %let rc=%sysfunc(dclose(&did));                                                                                                       
  %let rc=%sysfunc(filename(filrf));                                                                                                    
%mend drive; 
%sysmstoreclear;   /* Added by Pradip Muhuri*/

In [14]:
*Ex21_Access_Macro.sas;
libname Mylib 'C:\SASCourse\Compiled_Macros';
options mstored sasmstore=Mylib;
ods Excel file="C:\SASCourse\Week9\List_SAS_Files_x.xlsx"; 
%macro mexcel (start, stop); 
%DO num = &START %TO &STOP;
        ods excel options(sheet_name="Week&num");
        data _null_;                                                                                                                            
         file print;                                                                                                                 
         %drive(C:\SASCourse\Week&num,sas)                                                                                                                   
         run;  
%end;   
ods Excel close; 
%mend mexcel;
/* Create an Excel Workbook with two sheets, each listing 
  all SAS files from Week7 and Week8 folders, respectively*/
%mexcel (6,7)
